# Train Keras model and make performance plots

In [1]:
#Data Processing
import numpy as np
from numpy import loadtxt, expand_dims
import h5py
import sys

#ML model
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense,Flatten, Dropout, Activation, concatenate, BatchNormalization, GRU, Add, Conv1D, Conv2D, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model


from sklearn.metrics import roc_curve
from sklearn.metrics import auc

#Plot settings
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.ROOT)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 5

#Just to efficiently save plots
plot_dir = "plot/"
print("Saving plot to: ", plot_dir)
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y")
save_path = plot_dir + dt_string

Saving plot to:  plot/


## 1. Train model

In [3]:
#Load in the datasets for training and compiling the sample weights
with h5py.File('../data/trainingDataver3Pup.h5', 'r') as hf:
    dataset = hf["Training Data"][:]
with h5py.File('../data/sampleDataver3Pup.h5', 'r') as hf:
    sampleData = hf["Sample Data"][:]
    

#Separate datasets into inputs and outputs, expand the dimensions of the inputs to be used with Conv1D layers
X = dataset[:,0:len(dataset[0])-1]
X = expand_dims(X, axis=2)
y = dataset[:,len(dataset[0])-1]

print("X shape: ", X.shape)
print("y shape: ", y.shape)

#Establish the sample weights
thebins = np.linspace(0, 200, 100)
bkgPts = []
sigPts = []
for i in range(len(sampleData)):
    if y[i]==1:
        sigPts.append(sampleData[i][0])
    if y[i]==0:
        bkgPts.append(sampleData[i][0])
bkg_counts, binsbkg = np.histogram(bkgPts, bins=thebins)
sig_counts, binssig = np.histogram(sigPts, bins=thebins)
a = []
for i in range(len(bkg_counts)):
    tempSig = float(sig_counts[i])
    tempBkg = float(bkg_counts[i])
    if tempBkg!=0:
        a.append(tempSig/tempBkg)
    if tempBkg==0:
        a.append(0)
        
#Normalize the sample weights above a certain pT
for i in range(42,len(a)):
    a[i]=0.7
    
#Add in the sample weights, 1-to-1 correspondence with training data
#Sample weight of all signal events being equal to 1
#Sample weight of all background events being equal to the sig/bkg ratio at that jet's pT
weights = []
for i in range(len(sampleData)):
    if y[i]==1:
        weights.append(1)
    if y[i]==0:
        jetPt = sampleData[i][0]
        tempPt = int(jetPt/2)
        if tempPt>98:
            tempPt = 98
        weights.append(a[tempPt])

X shape:  (420000, 140, 1)
y shape:  (420000,)


In [4]:
#Define the model
model = Sequential()
model.add(Conv1D(filters=50, kernel_size=14, strides=14, activation='relu',input_shape=(len(dataset[0])-1,1)))
model.add(Conv1D(filters=50, kernel_size=1,activation='relu'))
model.add(GRU(100))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['binary_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 10, 50)            750       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 50)            2550      
_________________________________________________________________
gru (GRU)                    (None, 100)               45600     
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 54,471
Trainable params: 54,471
Non-trainable params: 0
____________________________________________________

In [5]:
#Train the network
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=5)

model.fit(X,y,
          epochs=50,
          batch_size=50,
          verbose=2,
          sample_weight=np.asarray(weights),
          validation_split=0.20,
          callbacks=[callback])

Epoch 1/50
6720/6720 - 82s - loss: 0.4882 - binary_accuracy: 0.6412 - val_loss: 0.4784 - val_binary_accuracy: 0.6537
Epoch 2/50
6720/6720 - 75s - loss: 0.4599 - binary_accuracy: 0.6587 - val_loss: 0.4412 - val_binary_accuracy: 0.6958
Epoch 3/50
6720/6720 - 76s - loss: 0.4338 - binary_accuracy: 0.6907 - val_loss: 0.4323 - val_binary_accuracy: 0.7012
Epoch 4/50
6720/6720 - 76s - loss: 0.4280 - binary_accuracy: 0.6934 - val_loss: 0.4317 - val_binary_accuracy: 0.7004
Epoch 5/50
6720/6720 - 75s - loss: 0.4249 - binary_accuracy: 0.6972 - val_loss: 0.4274 - val_binary_accuracy: 0.6721
Epoch 6/50
6720/6720 - 76s - loss: 0.4211 - binary_accuracy: 0.7023 - val_loss: 0.4264 - val_binary_accuracy: 0.6870
Epoch 7/50
6720/6720 - 75s - loss: 0.4180 - binary_accuracy: 0.7052 - val_loss: 0.4252 - val_binary_accuracy: 0.6824
Epoch 8/50
6720/6720 - 75s - loss: 0.4144 - binary_accuracy: 0.7072 - val_loss: 0.4296 - val_binary_accuracy: 0.6812
Epoch 9/50
6720/6720 - 67s - loss: 0.4101 - binary_accuracy: 0.7

In [5]:
model.save('saved_model/L1BTagModel_100GRUPup.h5')

## 2. Make performance plot for the model

In [ ]:
## See the other train_model_11